### Data Inspection

This notebook is used for the visualization and initial analysis of the EIS measurements

In [ ]:
%matplotlib widget 

from modules import eisplot as eisplot
from modules.eisplot import mpl
from modules.eisplot import plt


import numpy as np
import pandas as pd

## if you have installed latex and want to use it for plots, uncomment the following 3 lines
# mpl.rcParams.update({"text.usetex": True, "savefig.format": "pdf"})
# mpl.rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
# mpl.rc("text.latex", preamble=r"\usepackage{underscore}")

## safe figures e.g. with:
# plot_name = "custom_3D_plot"
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

The variable ```name_of_this_run``` is used to save and load the data

In [ ]:
name_of_this_run = "example_data"

In [ ]:
destination_filepath = r"./data/eis_datasets/" + name_of_this_run + ".parquet"
df = pd.read_parquet(destination_filepath)
destination_filepath = r"./data/key_lookup/key_lookup_" + name_of_this_run + ".parquet"
key_lookup_df = pd.read_parquet(destination_filepath)

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
abs_keys = key_lookup_df["EIS_Z_abs"].to_list()
phase_keys = key_lookup_df["EIS_Z_phase"].to_list()
re_keys = key_lookup_df["EIS_Z_Re"].to_list()
im_keys = key_lookup_df["EIS_Z_Im"].to_list()

Analysis of individual cells

In [ ]:
# # grab a cell by its name
cell_name = "LiFun_575166-01_002"
df_cell = df.loc[df.index.get_level_values("cell_ID") == cell_name]
# # or just grab the first one
# df_cell = df[np.isin(df.index.get_level_values(0), [df.first_valid_index()[0]])]
# # or sample e.g. 5 measurements
# df_cell = df.sample(n=5)

df_cell = df_cell.sort_values("SOC", ascending=True)
df_cell = df_cell.sort_values("Temperature", ascending=True)

There are two functions for typical EIS plots: `plot_nyquist_feature` and `plot_bode_feature`.

In [ ]:
fig, (ax1, ax2), cmap, cbar = eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature"
)

In [ ]:
fig, ax, cmap, cbar = eisplot.plot_nyquist_feature(
    df_cell, key_lookup_df, feature="Temperature"
)

Example of a custom figure:

In [ ]:
feature = "Temperature"
unit = "°C"

feature_values = df_cell[feature].to_numpy(dtype="float64")
c_min_value = np.min(feature_values)
c_max_value = np.max(feature_values)
norm = mpl.colors.Normalize(c_min_value, c_max_value)
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.turbo)
colors = cmap.to_rgba(feature_values)

size_scale = 30
size_offset = 10
alpha_scale = 0.75
alpha_offset = 0.25
x = df_cell["Voltage"].to_numpy(dtype="float64")
x = np.transpose(np.tile(x, (len(abs_keys), 1)))
y = df_cell[re_keys].to_numpy(dtype="float64") * 1000
z = df_cell[im_keys].to_numpy(dtype="float64") * -1000
colors = np.repeat(colors, len(abs_keys), axis=0)
sizes = df_cell["SOH"].to_numpy(dtype="float64") / 100
sizes = (
    np.clip(np.transpose(np.tile(sizes, (len(abs_keys), 1)) - 0.7), 0, None)
    * 10
    / 3
    * size_scale
    + size_offset
)
alphas = df_cell["SOC"].to_numpy(dtype="float64") / 100
colors[:, 3] = (
    np.transpose(np.tile(alphas, (len(abs_keys), 1))).reshape(-1) * alpha_scale
    + alpha_offset
)
markers = "o"

fig = plt.figure(figsize=(12 * eisplot.cm, 12 * eisplot.cm))
ax = fig.add_subplot(1, 1, 1, projection="3d")
cbar = fig.colorbar(cmap, ax=ax, location="left", fraction=0.046, pad=0.04)
cbar.set_label(feature + " in " + unit)
ax.scatter(xs=x, ys=y, zs=z, s=sizes, marker=markers, c=colors)

if mpl.rcParams["text.usetex"]:
    legend_elements = [
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 100 \%",
            alpha=1,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 0 \%",
            alpha=0.25,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 100 \%",
            alpha=0.5,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 70 \%",
            alpha=0.5,
            markersize=4,
            linestyle="",
        ),
    ]
    ax.set_xlabel("Voltage in V")
    ax.set_ylabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    ax.set_zlabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    legend_elements = [
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 100 %",
            alpha=1,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 0 %",
            alpha=0.25,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 100 %",
            alpha=0.5,
            markersize=8,
            linestyle="",
        ),
        plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 70 %",
            alpha=0.5,
            markersize=4,
            linestyle="",
        ),
    ]
    ax.set_xlabel("Voltage in V")
    ax.set_ylabel(r"Re(Z) in mΩ")
    ax.set_zlabel(r"Im(Z) in mΩ")

legend_elements = [
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoC = 100 %",
        alpha=1,
        markersize=8,
        linestyle="",
    ),
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoC = 0 %",
        alpha=0.25,
        markersize=8,
        linestyle="",
    ),
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoH = 100 %",
        alpha=0.5,
        markersize=8,
        linestyle="",
    ),
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoH = 70 %",
        alpha=0.5,
        markersize=4,
        linestyle="",
    ),
]
ax.legend(handles=legend_elements, loc="upper center", ncol=2, fontsize=8)

Before further plotting, lets get an overview of available parameters

In [ ]:
pd.set_option("display.max_columns", None)
df.head(1)
# pd.set_option('display.max_columns', 20)

In [ ]:
print([column for column in list(df) if "ECM" in column])
print([column for column in list(df) if "DRT" in column])
print([column for column in list(df) if "Bode" in column])
print([column for column in list(df) if "Nyquist" in column])
print(df.columns[0:15].values)

The easiest way is to directly use the pandas plot functions

In [ ]:
df.plot.scatter(
    x="SOC",
    y="ECM_Fit_RMSE",
    c="Temperature",
    colormap="turbo",
    figsize=(16 * eisplot.cm, 10 * eisplot.cm),
)

In [ ]:
fig, axs = plt.subplots(8, 8, figsize=(25 * eisplot.cm, 25 * eisplot.cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "Voltage",
            "SOH",
            "Temperature",
            "ECM_R0",
            "ECM_L0",
            "ECM_R1",
            "ECM_CPE1_0",
            "ECM_CPE1_1",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(9, 9, figsize=(25 * eisplot.cm, 25 * eisplot.cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "Voltage",
            "SOH",
            "Temperature",
            "ECM_R2",
            "ECM_CPE2_0",
            "ECM_CPE2_1",
            "ECM_R3",
            "ECM_CPE3_0",
            "ECM_CPE3_1",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(11, 11, figsize=(25 * eisplot.cm, 25 * eisplot.cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "Voltage",
            "SOH",
            "Temperature",
            "DRT_Peak_0_tau",
            "DRT_Peak_0_gamma",
            "DRT_Peak_1_tau",
            "DRT_Peak_1_gamma",
            "DRT_Peak_2_tau",
            "DRT_Peak_2_gamma",
            "DRT_Peak_3_tau",
            "DRT_Peak_3_gamma",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(7, 7, figsize=(20 * eisplot.cm, 20 * eisplot.cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "SOH",
            "Voltage",
            "Temperature",
            "Bode_Phase_Min_Freq_0",
            "Bode_Phase_Min_Value_0",
            "Bode_Phase_Min_Freq_1",
            "Bode_Phase_Min_Value_1",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
plt.tight_layout()

In [ ]:
corr = df[
    [
        "SOH",
        "ECM_R0",
        "DRT_Peak_0_gamma",
        "Bode_Phase_Min_Value_0",
        "Bode_Phase_Min_Value_1",
    ]
].corr()
corr.style.background_gradient(cmap="turbo", vmin=-1, vmax=1)

In [ ]:
# create a subset of the df
df_new = df[
    ["SOC", "Temperature", "ECM_R0", "DRT_Peak_0_tau", "Bode_Phase_Min_Freq_0"]
].copy()
# sort by the absolute scalar correlation of the values
sum_corr = abs(df_new.corr()).sum().sort_values(ascending=True).index.values
df_new = df_new[sum_corr]

# create a custom correlation matrix
g = eisplot.cor_matrix(df_new)

In [ ]:
# Configure the custom correlation matrix figure
g.figure.subplots_adjust(bottom=0.09)
g.figure.subplots_adjust(left=0.1, right=0.95)

g.axes[0, 0].set_ylabel("Normalised\n Probability Density", size=8)
g.axes[0, 0].set_yticklabels([])

name = "SoC in $\%$"
g.axes[4, 0].set_xlabel(name, size=8)

if mpl.rcParams["text.usetex"]:
    name = "$R_0$ in $\Omega$"
else:
    name = "R_0 in Omega"
g.axes[1, 0].set_ylabel(name, size=8)
g.axes[4, 1].set_xlabel(name, size=8)

name = "Tau of First\n Peak of DRT in s"
g.axes[2, 0].set_ylabel(name, size=8)
g.axes[4, 2].set_xlabel(name, size=8)

name = "Temperature\n in °C"
g.axes[3, 0].set_ylabel(name, size=8)
g.axes[4, 3].set_xlabel(name, size=8)

name = "Frequency of first\n Minimum of the Phase in Hz"
g.axes[4, 0].set_ylabel(name, size=8)
g.axes[4, 4].set_xlabel(name, size=8)

#### Further Bode and Nyquist plots

##### Bode

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(12 * eisplot.cm, 7 * eisplot.cm))
ax2 = ax1.inset_axes([0.5, 0.5, 0.47, 0.47])
df_cell = df_cell.sort_values("Temperature", ascending=False)
eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", fig=fig, ax1=ax1, ax2=ax2
)
ax2.remove()
ax1.set_autoscaley_on(True)
ax1.set_yscale("log")
fig.subplots_adjust(bottom=0.17, right=0.77)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(12 * eisplot.cm, 7 * eisplot.cm))
ax2 = ax1.inset_axes([0.5, 0.5, 0.47, 0.47])
eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", fig=fig, ax1=ax2, ax2=ax1
)
ax2.remove()
fig.subplots_adjust(bottom=0.17, right=0.77)

In [ ]:
fig, axs = plt.subplots(6, 1, sharex=True, figsize=(12 * eisplot.cm, 18 * eisplot.cm))
df = df.sort_values("SOH", ascending=True)
df = df.sort_values("SOC", ascending=True)
df = df.sort_values("Temperature", ascending=False)
eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    feature="Temperature",
    fig=fig,
    ax1=axs[0],
    ax2=axs[1],
    ax1_xlabel=False,
    ax2_xlabel=False,
)
df = df.sort_values("SOH", ascending=True)
df = df.sort_values("Temperature", ascending=False)
df = df.sort_values("SOC", ascending=True)
eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    feature="SOC",
    fig=fig,
    ax1=axs[2],
    ax2=axs[3],
    ax1_xlabel=False,
    ax2_xlabel=False,
    subplots_adjust=False,
)
df = df.sort_values("Temperature", ascending=False)
df = df.sort_values("SOC", ascending=True)
df = df.sort_values("SOH", ascending=True)
eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    feature="SOH",
    fig=fig,
    ax1=axs[4],
    ax2=axs[5],
    ax1_xlabel=False,
    subplots_adjust=False,
)

axs[0].set_autoscaley_on(True)
axs[0].set_yscale("log")
axs[2].set_autoscaley_on(True)
axs[2].set_yscale("log")
axs[4].set_autoscaley_on(True)
axs[4].set_yscale("log")

##### Nyquist

In [ ]:
fig, axs = plt.subplots(
    3, 1, sharey=True, sharex=True, figsize=(12 * eisplot.cm, 15 * eisplot.cm)
)

df = df.sort_values("SOH", ascending=True)
df = df.sort_values("SOC", ascending=True)
df = df.sort_values("Temperature", ascending=False)
eisplot.plot_nyquist_feature(
    df, key_lookup_df, feature="Temperature", fig=fig, ax=axs[0], ax_xlabel=False
)
df = df.sort_values("SOH", ascending=True)
df = df.sort_values("Temperature", ascending=False)
df = df.sort_values("SOC", ascending=True)
eisplot.plot_nyquist_feature(
    df, key_lookup_df, feature="SOC", fig=fig, ax=axs[1], ax_xlabel=False
)
df = df.sort_values("Temperature", ascending=False)
df = df.sort_values("SOC", ascending=True)
df = df.sort_values("SOH", ascending=True)
eisplot.plot_nyquist_feature(
    df, key_lookup_df, feature="SOH", fig=fig, ax=axs[2], subplots_adjust=False
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10 * eisplot.cm, 10 * eisplot.cm))

x_lim_main = [0, 6000]
y_lim_main = [50, -3500]
x_lim_inset = [20, 90]
y_lim_inset = [5, -25]
origin_inset = [0.1, 1.3]
inset_dimension = [0.9]


inset_width = y_lim_inset[0] - y_lim_inset[1]
inset_height = x_lim_inset[1] - x_lim_inset[0]
main_width = y_lim_main[0] - y_lim_main[1]
main_height = x_lim_main[1] - x_lim_main[0]

fig, ax, cmap, cbar = eisplot.plot_nyquist_feature(
    df_cell, key_lookup_df, feature="Temperature", fig=fig, ax=ax
)
ax.set_ylim(y_lim_main)
ax.set_xlim(x_lim_main)

axins = ax.inset_axes(
    np.concat([origin_inset, [inset_dimension[0]], [inset_dimension[0]]]),
    xlim=x_lim_inset,
    ylim=y_lim_inset,
)
fig, axins, cmap, cbar = eisplot.plot_nyquist_feature(
    df_cell,
    key_lookup_df,
    feature="Temperature",
    fig=fig,
    ax=axins,
    cmap=cmap,
    cbar=cbar,
    subplots_adjust=False,
    set_limits=False,
    set_ticks=False,
)

axins.set_aspect("equal", adjustable="datalim")
axins.set_xlabel("")
axins.set_ylabel("")
axins.set_xlim(x_lim_inset)
axins.set_facecolor("white")
ax.indicate_inset_zoom(
    axins, edgecolor=eisplot.rwth_colors.colors[("lime", 100)], alpha=0.5
)
fig.subplots_adjust(top=0.5, left=0.2)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10 * eisplot.cm, 15 * eisplot.cm), sharex=True)

x_lim_main = [0.01, 4500]
y_lim_main = [0, 6000]
x_lim_inset = [9, 101]
y_lim_inset = [25, 125]
origin_inset = [0.1, 1.3]
inset_dimension = [0.9]


inset_width = y_lim_inset[0] - y_lim_inset[1]
inset_height = x_lim_inset[1] - x_lim_inset[0]
main_width = y_lim_main[0] - y_lim_main[1]
main_height = x_lim_main[1] - x_lim_main[0]

fig, (ax[0], ax[1]), cmap, cbar = eisplot.plot_bode_feature(
    df_cell,
    key_lookup_df,
    feature="Temperature",
    fig=fig,
    ax1=ax[0],
    ax2=ax[1],
    ax1_xlabel=False,
)
ax[0].set_ylim(y_lim_main)
ax[0].set_xlim(x_lim_main)

ax_dummy = ax[0].inset_axes([0.5, 0.5, 0.47, 0.47])
axins = ax[0].inset_axes(
    np.concat([origin_inset, [inset_dimension[0]], [inset_dimension[0]]]),
    xlim=x_lim_inset,
    ylim=y_lim_inset,
)
fig, (axins, ax_dummy), cmap, cbar = eisplot.plot_bode_feature(
    df_cell,
    key_lookup_df,
    feature="Temperature",
    fig=fig,
    ax1=axins,
    ax2=ax_dummy,
    cmap=cmap,
    cbar=cbar,
    subplots_adjust=False,
    set_limits=False,
    set_ticks=False,
)
ax_dummy.remove()

axins.set_xlabel("")
axins.set_ylabel("")
axins.set_xlim(x_lim_inset)
axins.set_facecolor("white")
ax[0].indicate_inset_zoom(
    axins, edgecolor=eisplot.rwth_colors.colors[("lime", 100)], alpha=0.5
)

fig.subplots_adjust(top=0.65, left=0.15, right=0.9)
cbar_label = cbar.ax.get_ylabel()
cbar.remove()
cbar = plt.colorbar(mappable=cmap, ax=ax)
cbar.set_label(cbar_label)

Let's have a look at the average EIS measurement

In [ ]:
abs_value_mean = df.loc[:, abs_keys].to_numpy(dtype="float64").mean(axis=0)
phase_value_mean = df.loc[:, phase_keys].to_numpy(dtype="float64").mean(axis=0)

impedance_mean = abs_value_mean * np.exp(1j * phase_value_mean)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14 * eisplot.cm, 6 * eisplot.cm))
ax.plot(np.real(impedance_mean) * 1000, np.imag(impedance_mean) * 1000)
ax.grid()
ax.set_aspect("equal", "box")
if mpl.rcParams["text.usetex"]:
    ax.set_xlabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    ax.set_ylabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    ax.set_xlabel(r"Re(Z) in mΩ")
    ax.set_ylabel(r"Im(Z) in mΩ")
ax.invert_yaxis()

#### Bode and Nyquist of reduced amout of measurements

In [ ]:
fig, ax, cmap, cbar = eisplot.plot_nyquist_feature(
    df_cell, key_lookup_df, feature="Temperature", reduce=True, nr_intervals=8
)

In [ ]:
fig, ax, cmap, cbar = eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", reduce=True, nr_intervals=8
)

#### Bode and Nyquist with highlighted frequencies

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
highlight_freqs = [
    frequencies[0],
    frequencies[5],
    frequencies[14],
    frequencies[30],
    frequencies[36],
]

fig, ax, cmap, cbar = eisplot.plot_bode_feature(
    df_cell,
    key_lookup_df,
    "Temperature",
    highlight_freqs=highlight_freqs,
    reduce=True,
    nr_intervals=8,
)

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
highlight_freqs = [
    frequencies[0],
    frequencies[5],
    frequencies[14],
    frequencies[30],
    frequencies[36],
]

fig, axs, cmap, cbar = eisplot.plot_nyquist_feature(
    df_cell,
    key_lookup_df,
    "Temperature",
    highlight_freqs=highlight_freqs,
    reduce=True,
    nr_intervals=8,
)

#### Bode and Nyquist with highlighted extrema

In [ ]:
df = df.sort_values("Temperature")

In [ ]:
highlight_df_columns = [
    ["Nyquist_Min_Real_Value_0", "Nyquist_Min_Imag_Value_0"],
    ["Nyquist_Max_Real_Value_0", "Nyquist_Max_Imag_Value_0"],
]

fig, axs, cmap, cbar = eisplot.plot_nyquist_feature(
    df,
    key_lookup_df,
    "Temperature",
    reduce=True,
    nr_intervals=9,
    highlight_df_columns=highlight_df_columns,
)

In [ ]:
fig, axs = plt.subplots(
    2, 1, sharey=False, sharex=True, figsize=(12 * eisplot.cm, 10 * eisplot.cm)
)

highlight_df_columns = [
    ["Bode_Phase_Min_Freq_0", "Bode_Phase_Min_Value_0"],
    ["Bode_Phase_Max_Freq_0", "Bode_Phase_Max_Value_0"],
]

fig, axs, cmap, cbar = eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    "Temperature",
    fig=fig,
    ax1=axs[0],
    ax2=axs[1],
    reduce=True,
    nr_intervals=8,
    highlight_df_columns=highlight_df_columns,
)

### 1 kHz

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16 * eisplot.cm, 8 * eisplot.cm))

# define frequency
frequency = 1000
frequency_label = [
    key_lookup_df["EIS_Z_abs"].loc[
        np.argmin(np.abs(key_lookup_df["frequency"].values - frequency))
    ]
]

# color
min_value_color = df_cell["SOH"].min()
max_value_color = df_cell["SOH"].max()
norm = mpl.colors.Normalize(min_value_color, max_value_color)
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.turbo)
cmap.set_array([])
colors = cmap.to_rgba(df_cell["SOH"].to_numpy("float64"))

# marker size
min_value_marker = df_cell["SOC"].min()
max_value_marker = df_cell["SOC"].max()
h, edges = np.histogram(df_cell["SOC"], bins="auto")
hist_center = (edges[:-1] + edges[1:]) / 2
marker_size = (
    5 + (hist_center - min_value_marker) / (max_value_marker - min_value_marker) * 5
)
marker_groups = hist_center.astype("str")
marker_dict = dict(zip(marker_groups, marker_size))
marker_sizes = (
    pd.cut(np.abs(df_cell.loc[:, "SOC"]), edges, labels=marker_groups)
    .map(marker_dict)
    .values.astype(float)
)

# marker shape
markers = np.array(["x" for x in range(len(df_cell))])
# markers[discharge_mask] = 'v'
# markers[charge_mask] = '^'

for x, y, c, m, ms in zip(
    df_cell["Temperature"].values,
    df_cell[frequency_label].values * 1000,
    colors,
    markers,
    marker_sizes,
):
    axs.plot(x, y, color=c, marker=m, markersize=ms, linestyle="")

cbar = fig.colorbar(cmap, ax=axs, location="right", fraction=0.046, pad=0.04)
cbar.set_label("SOH in %")
axs.set_xlabel("Temperature in °C")
axs.set_ylabel("|Z| @ 1 kHz in mΩ")

legend_elements = [
    mpl.lines.Line2D(
        [0],
        [0],
        marker="x",
        linestyle="",
        color=eisplot.rwth_colors.colors[("black", 50)],
        label=str(np.round(min_value_marker, 2)) + " % SOC",
        alpha=1,
        markersize=np.min(marker_size),
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="x",
        linestyle="",
        color=eisplot.rwth_colors.colors[("black", 50)],
        label=str(np.round(max_value_marker, 2)) + " % SOC",
        alpha=1,
        markersize=np.max(marker_size),
    ),
]
axs.legend(handles=legend_elements, loc="best", scatterpoints=1, ncol=1)

# axs.set_ylim(y_limits[cell_idx])
axs.grid()
fig.tight_layout()